<a href="https://colab.research.google.com/github/bartekb4/ObjectDetectionINZ/blob/main/Model_detekcyjny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
 
drive.mount('/gdrive')
# the project's folder
%cd /gdrive/'My Drive'/tf/tensorflow

In [ ]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv

import re
import os
import io
import glob
import shutil
import urllib.request
import tarfile
import xml.etree.ElementTree as ET

import tensorflow as tf
import cv2 

from PIL import Image
from collections import namedtuple, OrderedDict

from google.colab import files

In [ ]:
!apt-get install -qq protobuf-compiler python-pil python-tk
!pip install -qq Cython contextlib2 pillow matplotlib pycocotools
!pip install cython
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI
!pip install numpy==1.17.4

In [ ]:
%cd /gdrive/My Drive/tf/tensorflow/models/research
!protoc object_detection/protos/*.proto --python_out=.


In [ ]:
import os
pwd = os.getcwd()
os.environ['PYTHONPATH'] += f':{pwd}:{pwd}/slim'

In [ ]:
%cd /gdrive/My Drive/tf/tensorflow/models/research/
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

In [ ]:
%cd /gdrive/My Drive/tf/tensorflow/workspace/training_demo/exported_models
import time

PATH_TO_SAVED_MODEL = "my_model/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

In [ ]:
%cd /gdrive/My Drive/tf/tensorflow/workspace/testing/test/

In [ ]:
import os


path = '/gdrive/My Drive/tf/tensorflow/workspace/testing/test/'
IMAGE_PATHS = os.listdir(path)
print(IMAGE_PATHS)


In [ ]:
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util
category_index = label_map_util.create_category_index_from_labelmap('/gdrive/My Drive/tf/tensorflow/workspace/training_demo/annotations/label_map.pbtxt', use_display_name=True)
#from object_detection.utils import visualization_utils as viz_utils
%cd /gdrive/My Drive/tf/tensorflow/workspace/testing/test/
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
#warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings
path="/gdrive/My Drive/tf/tensorflow/workspace/testing/test/"
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          min_score_thresh=.4,
          use_normalized_coordinates=True,
          line_thickness=1)

    plt.figure()
    #print(detections['detection_boxes']) 
    #plt.imshow(image_np_with_detections)
    print('Done')
    #display(Image.fromarray(image_np_with_detections))
plt.show()

# sphinx_gallery_thumbnail_number = 2